# Single Node Single GPU Training

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
from tensorflow_snowflake_class import SnowflakeDataset

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import dask


/srv/conda/envs/saturn/lib/python3.9/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [3]:
model_params = {'n_epochs': 3, 
                'base_lr': .02,
               'batchsize': 32}

wbargs = {**model_params,
    'classes':10,
    'Notes':"tf_baseline",
    'Tags': ['single', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet"}

In [6]:
def train_model_fit(n_epochs, base_lr, batchsize):
    
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        classes=10)
        
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_rnd')
    config=wbargs
    
    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/data/clothing-dataset-small/train',
        seed=123,
        image_size=(224,224),
        batch_size=32).prefetch(2)

    
    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels

    outputs = model.fit(
        train_ds, 
        verbose = 1, 
        epochs=n_epochs, 
        callbacks=[WandbCallback()]
    )
    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras/')

In [ ]:
tester = train_model_fit(**model_params)

Found 3068 files belonging to 10 classes.
Epoch 1/3
96/96 [==============================] - 35s 294ms/step - loss: 6.6139 - accuracy: 0.2034
Epoch 2/3
96/96 [==============================] - 28s 286ms/step - loss: 2.1197 - accuracy: 0.2606
Epoch 3/3
96/96 [==============================] - 28s 287ms/step - loss: 2.1275 - accuracy: 0.2564


epoch,2
loss,2.14592
accuracy,0.25913
_runtime,93
_timestamp,1629229423
_step,2


epoch,▁▅█
loss,█▁▁
accuracy,▁██
_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█
